In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../src')

In [2]:
from get_data import load_history, load_current, combine_history, process_y, load_bios, load_team_data
load_history()
load_current()
# combine_history()
# load_team_data()
# load_bios()
# process_y()


https://moneypuck.com/moneypuck/playerData/teamPlayerGameByGame/2021/regular/skaters/


/home/jupyter/mehhala-fantasy/notebooks/../src/get_data.py:17: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 17 of the file /home/jupyter/mehhala-fantasy/notebooks/../src/get_data.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(r.text)


https://moneypuck.com/moneypuck/playerData/teamPlayerGameByGame/2022/regular/skaters/
https://moneypuck.com/moneypuck/playerData/teamPlayerGameByGame/2023/regular/skaters/


KeyboardInterrupt: 

In [ ]:
from yahoo_utils import get_games_by_week
current_schedule = get_games_by_week(427)

In [ ]:
from process_data import get_rest_of_season_player_stats, get_rest_of_season_stats, PRED_COLS
from model_training import get_simple_pipelines, load_player_feature_map


X, y = get_rest_of_season_player_stats('skater')
y = y.loc[X.index, PRED_COLS['skater']]

In [ ]:
X_g, y_g = get_rest_of_season_player_stats('goalie')
y_g = y_g.loc[X_g.index, PRED_COLS['goalie']]

In [ ]:
pipelines = get_simple_pipelines((X, y), (X_g, y_g))

In [ ]:
for col in PRED_COLS['skater']:    
    preds[col] = pipelines['skater'][col].predict(X[skaters_p_feats[col][0]])
preds_df = pd.DataFrame(preds, index=X.index)

gl_p_cols = list(set([c for s in goalies_p_feats.values() for c in s[0]]))
for col in PRED_COLS['goalie']:
    goalie_preds[col] = pipelines['goalie'][col].predict(X_g[goalies_p_feats[col][0]])
goalie_preds_df = pd.DataFrame(goalie_preds, index=X_g.index)



games_df = pd.DataFrame([dict(week=week, **game) for week, games in current_schedule.items() for game in games])

preds_df = preds_df.merge(games_df[['gameId','week']], on='gameId', how='left')\
    .set_index(preds_df.index)
goalie_preds_df = goalie_preds_df.merge(games_df[['gameId','week']], on='gameId', how='left')\
    .set_index(goalie_preds_df.index)

preds_df = pd.concat([preds_df, goalie_preds_df], axis=0)
weekly_preds = preds_df[preds_df['week'] > 0].set_index('week', append=True)
weekly_preds = weekly_preds.drop('gameId', axis=1)

weekly_preds.to_csv('data/2324_preds_simple.csv')

In [ ]:
preds = {}
goalie_preds = {}

for col in PRED_COLS['skater']:    
    preds[col] = pipelines['skater'][col].predict(skater_latest.dropna()[skaters_p_feats[col][0]])
preds_df = pd.DataFrame(preds, index=skater_latest.dropna().index)

gl_p_cols = list(set([c for s in goalies_p_feats.values() for c in s[0]]))
for col in PRED_COLS['goalie']:
    goalie_preds[col] = pipelines['goalie'][col].predict(goalie_latest.dropna()[goalies_p_feats[col][0]])
goalie_preds_df = pd.DataFrame(goalie_preds, index=goalie_latest.dropna().index)

preds_df = pd.concat([preds_df, goalie_preds_df], axis=0)
weekly_preds.to_csv('data/latest_preds.csv')